Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Onnx Pipeline

This repository shows how to deploy and use Onnx pipeline with dockers.

# Prerequisites

Pull dockers. It should take several minutes.

In [74]:
!sh build.sh

Authenticating with existing credentials...
Login Succeeded
Using default tag: latest
latest: Pulling from onnx-converter
Digest: sha256:080d344ba335d2131ae15769dd8daf149acaa5a1f122df998c2c7223231a62f4
Status: Image is up to date for ziylregistry.azurecr.io/onnx-converter:latest
Using default tag: latest
latest: Pulling from create-input
Digest: sha256:cb94537e14345c9f0a03014566006658b030c5fed66f35bb511471838c733df0
Status: Image is up to date for ziylregistry.azurecr.io/create-input:latest
Using default tag: latest
latest: Pulling from perf_test
Digest: sha256:0b93e6a1d4e4cd5e0057cf503fce53b8702d2445252b7837e844e50752d2a369
Status: Image is up to date for ziylregistry.azurecr.io/perf_test:latest


# Convert model to ONNX

In [75]:
!docker run -v %CD%\model_240000:/mnt/model_240000 ziylregistry.azurecr.io/onnx-converter --model model_240000 --output_onnx_path /mnt/output/elmo.onnx --model_type tensorflow

2019-05-28 23:18:13.396371: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/site-packages/tf2onnx/convert.py", line 135, in <module>
    main()
  File "/usr/local/lib/python3.6/site-packages/tf2onnx/convert.py", line 99, in main
    graph_def, inputs, outputs = loader.from_saved_model(args.saved_model, args.inputs, args.outputs)
  File "/usr/local/lib/python3.6/site-packages/tf2onnx/loader.py", line 87, in from_saved_model
    meta_graph_def = tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.SERVING], model_path)
  File "/usr/local/lib/python3.6/site-packages/tensorflow/python/saved_model/loader_impl.py", line 196, 

## Run parameters

(1) model: string

Required. The path of the model that needs to be converted.

(2) output_onnx_path: string

Required. The path to store the converted onnx model. Should end with ".onnx". e.g. output.onnx

(3) model_type: string

Required. The name of original model framework.

Available types are caffe, cntk, coreml, keras, libsvm, mxnet, scikit-learn, tensorflow and pytorch.

(4) model_inputs: string

Optional. The model's input names. Required for tensorflow frozen models and checkpoints.

(5) model_outputs: string

Optional. The model's output names. Required for tensorflow frozen models checkpoints.

(6) model_params: string

Optional. The params of the model if needed.

(7) model_input_shapes: list of tuple

Optional. List of tuples. The input shape(s) of the model. Each dimension separated by ','.

(8) target_opset: int

Optional. Specifies the opset for ONNX, for example, 7 for ONNX 1.2, and 8 for ONNX 1.3. Defaults to 7.


# Generate-Input image

In [76]:
!docker run -v %CD%\deepthink.onnx:/mnt/deepthink.onnx ziylregistry.azurecr.io/create-input --model /mnt/deepthink.onnx

/mnt/deepthink.onnx inputs: 
input name: long_MRC_2/passage_mask:0, shape: [32, None, 40], type: tensor(float)
input name: long_MRC_2/passage:0, shape: [32, None, 40], type: tensor(int32)
input name: long_MRC_2/target_mask:0, shape: [32], type: tensor(int32)
input name: long_MRC_2/start_only_label:0, shape: [32, None], type: tensor(float)
input name: long_MRC_2/sent_passage_mask:0, shape: [32, None], type: tensor(float)
input name: long_MRC_2/ques_mask:0, shape: [None, 40], type: tensor(float)
input name: long_MRC_2/ques_input:0, shape: [None, 40], type: tensor(int32)
Randomized input .pb file generated at  /mnt/test_data_set_0


## Run parameters

(1) model: string
    
Required. The path of the model that needs to be converted.


# Performance test tool

In [77]:
!docker run -v model:/mnt/model ziylregistry.azurecr.io/perf_test /mnt/model /mnt/result

Cores:  1

/home/artr/repo/onnxruntime/include/onnxruntime/core/session/onnxruntime_cxx_api.h:127 OrtCreateSession /home/artr/repo/onnxruntime/onnxruntime/core/providers/cuda/cuda_call.cc:97 bool onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*) [with ERRTYPE = cudaError; bool THRW = true] /home/artr/repo/onnxruntime/onnxruntime/core/providers/cuda/cuda_call.cc:91 bool onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*) [with ERRTYPE = cudaError; bool THRW = true] CUDA failure 35: CUDA driver version is insufficient for CUDA runtime version ; GPU=32638 ; hostname=5645bc30e758 ; expr=cudaSetDevice(device_id_); 
Stacktrace:

Stacktrace:



Setting thread pool size to 0


/home/artr/repo/onnxruntime/include/onnxruntime/core/session/onnxruntime_cxx_api.h:127 OrtCreateSession /home/artr/repo/onnxruntime/onnxruntime/core/providers/cuda/cuda_call.cc:97 bool onnxruntime::CudaCall(ERRTYPE, const char*, const char*, ERRTYPE, const char*) [w